In [1]:
import os
import pickle as pkl
from typing import List, Optional

import fire

from llama import Llama, Dialog

from tqdm.auto import tqdm
from time import sleep
from datasets import load_dataset, DatasetDict
from torch.utils.data import DataLoader, Dataset
import pickle as pkl
import os

# os.environ['CUDA_VISIBLE_DEVICES']="1,2"
import pandas as pd
import nltk
nltk.download('punkt')
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
from datasets import load_dataset, DatasetDict, load_from_disk
from torch.utils.data import DataLoader, Dataset
import numpy as np
from tqdm import tqdm
import pandas as pd
from nltk import sent_tokenize
import math, re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from torchmetrics.text.rouge import ROUGEScore
from transformers import Trainer, TrainingArguments, pipeline
import argparse
import evaluate
# from styleformer import Styleformer
import warnings
warnings.filterwarnings("ignore")
import copy
import multiprocessing
import pickle as pkl
import openai
from dotenv import load_dotenv
import os
from time import sleep

/home/haskari/miniconda3/envs/acl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/haskari/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[2024-01-23 20:39:24,261] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
with open("xsum/0.pkl", 'rb') as f:
    file=pkl.load(f)

In [3]:
print(file)

['Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation. Workers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders. The Welsh Government said more people than ever were getting help to address housing problems.  Prison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered. However, the same could not be said for men, the charity said, because issues which often affect them, such as post traumatic stress disorder or drug dependency, were often viewed as less of a priority.  "There\'s a desperate need for it, finding suitable accommodation for those leaving prison there is just a lack of it everywhere," he said. "It could take six months to a year, without a lot of help they could be on the streets for six months. "W

In [4]:
dataset = load_dataset('reddit_tifu', 'long')
article_key = 'documents'
summary_key = 'tldr'
# 80% train, 20% test + validation
train_testvalid = dataset['train'].train_test_split(test_size=0.2, seed=42)
# Split the 20% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)
# gather everyone if you want to have a single DatasetDict
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

name='reddit'

In [6]:
len(dataset['test'])

4214

In [11]:
dataset['test'][0]['documents']

'well this was around 9 years ago when i was a 2nd grader. so basically it was some kids birthday and since it was primary school most kids brought cupcakes and the teacher would pass it out to the class. me being the hungry little shit that i was i was desperately in the mood for some vanilla cupcakes but the teacher decided to pass out the chocolate ones first. \n\nafter she had finished passing out the chocolate ones she picks up the vanilla ones and says, "raise your hand if you want vanilla!". at that moment i was talking to a friend and it took me a bit to process the fact that she just asked who wanted vanilla. i proceed to raise my hand and it turns out the cupcakes were right above me. i knocked the cupcakes out of her hand and they spilled  all over her. she got bat-shit angry and begins to make a scene in the middle of the whole cafeteria. she explained how i "ruined her brand new shoes" and the bitch even asked me if i had an allowance so i could pay for her shoes. \n\nshe 

In [14]:
dataset['test'][0]['paraphrased']='lol'

In [17]:
paraphrased_articles=['lol']*4214

In [20]:
dataset=dataset['test']

In [24]:
dataset = dataset.remove_columns("documents")

In [26]:
dataset = dataset.add_column("documents", paraphrased_articles)

In [10]:
def tokenize(example):
    example["tokenized_document"] = nltk.sent_tokenize(example[article_key])
    example["tokenized_summary"] = nltk.sent_tokenize(example[summary_key])
   # example['segment_idxs'] = generate_n_segments(example["article"]) #NEW
    return example

def tokenize_news(example):
    example["tokenized_document"] = nltk.sent_tokenize(example[article_key])
    example["tokenized_summary"] = nltk.sent_tokenize(example[summary_key][0][article_key])
    #example['segment_idxs'] = generate_n_segments(example["article"]) #NEW
    return example

In [ ]:
def generate_paraphrased_articles(generator,dataset, dataset_name, max_gen_len,temperature,top_p,batch_size=1):
    
    if dataset_name!='news':
        dataset = dataset.map(tokenize, num_proc=multiprocessing.cpu_count())
    else:
        dataset = dataset.map(tokenize_news, num_proc=multiprocessing.cpu_count())
        
    articles = dataset["tokenized_document"]
    highlights = dataset["tokenized_summary"]